In [3]:
from pyspark.sql import SparkSession

from pyspark.sql import functions as f
#from pyspark.sql.functions import  sum, avg, min, max,  mean
from pyspark.sql.types import *
from pyspark.sql.functions import countDistinct, avg, stddev
from pyspark.sql.functions import col, monotonically_increasing_id


from ut_spark import ut_spark 
from ut_spr import ut_spr as ut1

spark =SparkSession.\
        builder. \
        appName("Aggregation"). \
        master("local4]").\
        getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Could not parse Master URL: 'local4]'
	at org.apache.spark.SparkContext$.org$apache$spark$SparkContext$$createTaskScheduler(SparkContext.scala:2924)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:548)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(Unknown Source)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(Unknown Source)
	at java.lang.reflect.Constructor.newInstance(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:238)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.comm

#### Read flight,  plane, airport, carrier data from raw files to DataFrames 

In [54]:
def read_flight():
    """ Read  flight-on=time data '2008.csv.bz2' to a spark DataFrame
        Returns: spark DataFrame
    """
    ft_fields = [
        ('Year', 'int'),
        ('Month', 'int'),
        ('DayofMonth', 'int'),
        ('DayOfWeek', 'int'),
        ('DepTime', 'int'),
        ('CRSDepTime', 'int'),
        ('ArrTime', 'int'),
        ('CRSArrTime', 'int'),
        ('UniqueCarrier', 'str'),
        ('FlightNum', 'str'),
        ('TailNum', 'str'),
        ('ActualElapsedTime', 'int'),
        ('CRSElapsedTime', 'int'),
        ('AirTime', 'int'),
        ('ArrDelay', 'int'),
        ('DepDelay', 'int'),
        ('Origin', 'str'),
        ('Dest', 'str'),
        ('Distance', 'int'),
        ('TaxiIn', 'int'),
        ('TaxiOut', 'int'),
        ('Cancelled', 'int'),
        ('CancellationCode', 'str'),
        ('Diverted', 'str'),
        ('CarrierDelay', 'int'),
        ('WeatherDelay', 'int'),
        ('NASDelay', 'int'),
        ('SecurityDelay', 'int'),
        ('LateAircraftDelay', 'int')
    ]

    ft_schema = ut_spark.list_to_schema(ft_fields)

    f_folder = 'C:\\demo\\capstone\\raw_data\\'
    f_name = '2008.csv.bz2'
    f_path = f_folder + f_name

    return spark.read \
        .option("header", "true") \
        .schema(ft_schema) \
        .csv(f_path)

df_flight=read_flight().withColumn('ROW_ID', monotonically_increasing_id())

In [3]:
from ut_spark import ut_spark as ut2 
def read_plane():
    """ Read  plane data  to a spark DataFrame """

    f_folder = 'C:\\demo\\capstone\\raw_data\\'
    f_name ='plane-data.csv'
    f_path= f_folder+ f_name

    return ut2.load_df_csv(spark, f_path)

df_plane= read_plane().withColumn('ROW_ID', monotonically_increasing_id())

In [4]:
def read_airport():
    """ Read  airport data  to a spark DataFrame """
    
    f_folder = 'C:\\demo\\capstone\\raw_data\\'
    f_name ='airports.csv'
    f_path = f_folder + f_name

    return spark.read \
        .options(header="true") \
        .option("inferSchema", "true") \
        .csv(f_path)

df_airport = read_airport().withColumn('ROW_ID', monotonically_increasing_id())

In [5]:
def read_carrier():
    """ Read  carrier data  to a spark DataFrame and add ROW_ID column to it """

    f_folder = 'C:\\demo\\capstone\\raw_data\\'
    f_name ='carriers.csv'
    f_path = f_folder + f_name

    return spark.read \
        .options(header="true") \
        .option("inferSchema", "true") \
        .csv(f_path)

#read carrier data and add ROW_ID to it
df_carrier = read_carrier().withColumn('ROW_ID', monotonically_increasing_id())

In [11]:
df_airport.printSchema()

root
 |-- iata: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- ROW_ID: long (nullable = false)



####  Check referential integrity between datasets


In [73]:
def cnt_leftouter_null(df1, df2, join_col1, join_col2):
   """To get the number of reords of the child dataset, whose joining column value not matching  the  parent dataset column 
      Args:
        df1: child dataframe
        df2: parent dataframe
        join_col1: joining column of child dataframe
        join_col2: joining column prent dataframe

      Returns: reords of records not matching
   """
    
   cnt = df1.join(df2, df1[join_col1] == df2[join_col2],"leftouter").filter(df1[join_col2].isNull()).count()
   print("number of records with missing values".format(cnt))
   return cnt

In [47]:
# Check 'Origin' column of df_flight
cnt_mis_origin=df_flight.join(df_airport,df_flight.Origin ==  df_airport.iata,"leftouter").filter(df_airport["iata"].isNull()).count()
    

In [43]:
# check 'Dest' column  of  df_flight
cnt_mis_dest=df_flight.join(df_airport,df_flight.Dest ==  df_airport.iata,"leftouter").filter(df_airport["iata"].isNull()).count()


In [36]:
# check 'TailNum' column  of  df_flight
cnt_mis_tailnum=cnt_leftouter_null(df_flight, df_plane, "TailNum", "tailnum")


In [52]:
print ("cnt_mis_origin= {}, cnt_mis_dest= {}, cnt_mis_tailnum= {}".format(cnt_mis_origin, cnt_mis_dest, cnt_mis_tailnum))

cnt_mis_origin=0, cnt_mis_dest=0, cnt_mis_tailnum=42452
